In [ ]:
%load_ext autoreload
%autoreload 2

import sys 
sys.path.insert(0, '..')
import jefpy as jp
import numpy as np
import matplotlib.pyplot as plt

%matplotlib qt

## Static dipole

In [ ]:
dipole = jp.ElectricDipole(location=[0, 1, 3], p=[0, 0, 1])
print(dipole.B([2, 1, 5]))
print(dipole.E([2, 1, 5]))

Requesting the field at different times gives the same answer for this static example. 

In [ ]:
print(dipole.E([2, 1, 5], [0.0, 1.0, 2.0]))

A source can be moving through space. Use a call-back for the location.

In [ ]:
def location(t):
    return np.array([1, 2, 3]) * np.cos(t)

dipole = jp.ElectricDipole(location=location, p=[0, 0, 1])

In [ ]:
print(dipole.E([10, 10, -10], np.linspace(0, 3, 10)))

In [ ]:
print(dipole.E([[10, 10, -10], [5, 5, 5]], np.linspace(0, 0.5, 5)).shape)

Note the polymorphism for the shapes of r and t. The returned fields have the following structure: The first dimension is time if time is an array; the last dimension is x, y, z; the middle dimensions are spatial variation. 

### Surface

In [ ]:
xrange, yrange = [-100.0, 100.0], [-50.0, 50.0]
surface_settings = {
    'x': np.linspace(*xrange, 10),
    'y': np.linspace(*yrange, 5),
    'z': 0}
surface = jp.Surface.cartesian(**surface_settings)

print(surface.XYZ.shape)
print(surface.U.shape)

In [ ]:
dipole = jp.ElectricDipole(location=[0, 0, 0.0], p=[0, 1, 0])

xrange, yrange = [-100.0, 100.0], [-100.0, 100.0]
surface_settings = {
    'x': np.linspace(*xrange, 20),
    'y': np.linspace(*yrange, 20),
    'z': 0}
surface = jp.Surface.cartesian(**surface_settings)

E = dipole.E(surface.XYZ)
Ex = E[..., 0].T
Ey = E[..., 1].T

plt.figure()
plt.streamplot(surface.u, surface.v, Ex, Ey)

## Oscillating dipole

In [ ]:
dipole = jp.ElectricDipole.oscillator()

In [ ]:
dipole_settings = {
    'power': 1.0,
    'orientation': [0, 0, 1],
    'freq': 200e6,
    'location': [0, 0, 0]}
dipole = jp.ElectricDipole.oscillator(**dipole_settings)

In [ ]:
print(dipole.B([1.0, 1, 1], 0.0))
print(dipole.B([1.0, 1, 1], 1e-10))
print(dipole.E([1.0, 1, 1], 0.0))
print(dipole.E([1.0, 1, 1], 1e-10))

We can have two dipoles (the second one moving in space)

In [ ]:
# Dipole source 2
dipole_settings = {
    'power': 1.0,
    'orientation': [0, 0, 1],
    'freq': 270e6,
    'location': lambda t: np.array([1, 0, 1]) * t }

dipole_2 = jp.ElectricDipole.oscillator(**dipole_settings)

In [ ]:
dipole.E([5.0, 5.0, 5.0], 0.0) + dipole_2.E([5.0, 5.0, 5.0], np.linspace(0, 0.5, 5))

## Source Collections
Multiple sources can be combined in a SourceCollection to achieve simpler interfacing. 

In [ ]:
sources = jp.SourceCollection({'dip 1': dipole, 'dip 2': dipole_2})
sources

In [ ]:
sources.E([5.0, 5.0, 5.0], np.linspace(0, 0.5, 5))

Source collections are the main datastructure of jefpy and can be created in many intuitive ways:

In [ ]:
from pprint import pprint

sources = jp.SourceCollection()
sources['source 1'] = dipole
sources['source 2'] = dipole_2
pprint(sources)
print(" ----  ")


sources = jp.SourceCollection((dipole, dipole_2))
pprint(sources)
print(" ----  ")

sources = dipole + dipole_2
pprint(sources)
print(" ----  ")

sources = sources + dipole_2
pprint(sources)
print(" ----  ")

sources['named source'] = jp.MagneticDipole()
pprint(sources)
print(" ----  ")

source_col_1 = jp.ElectricDipole() + jp.MagneticDipole()
source_col_2 = jp.ElectricDipole() + jp.MagneticDipole()
sources = source_col_1 + source_col_2
pprint(sources)

## Observers
If you have a fixed set of observation location it can be convenient to define an observer.

In [ ]:
r1 = [-5.0, -5.0, -4.0]
r2 = [5.0, 5.0, 5.0]
r3 = [5.0, 5.0, 6.0]
r4 = [5.0, 5.0, 7.0]

sources = jp.SourceCollection({'dip 1': dipole, 'dip 2': dipole_2})
observer = jp.Observer((r1, r2, r3, r4), sources)

In [ ]:
observer.E(0.2)

In [ ]:
observer.B(0.1)

It is actually possible to define it a little shorter

In [ ]:
observer = jp.Observer((r1, r2, r3, r4), {'dip 1': dipole, 'dip 2': dipole_2})

or even shorter

In [ ]:
observer = jp.Observer((r1, r2, r3, r4), (dipole, dipole_2))

Some basic time based plotting.